### Importação de bibliotecas e sessão do pyspark:

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import functools

spark = SparkSession.builder.getOrCreate()

### Criando esquemas para as tabelas:
* Definindo o cabeçalho e os tipos de dados de cada coluna na tabela de clientes.
* Lendo arquivos CSVs a partir do esquema definido.

In [ ]:
schema = StructType() \
    .add('Id', IntegerType(),True) \
    .add('Nome', StringType(),True) \
    .add('Email', StringType(),True) \
    .add('Data_cadastro', TimestampType(),True) \
    .add('Telefone', StringType(),True)

df_clients = spark.read.format('csv') \
    .option('inferSchema', True) \
    .schema(schema) \
    .load('./data/clients/', delimiter=';')

df_clients.show()

* Definindo o cabeçalho e os tipos de dados de cada coluna na tabela de Transações.
* Lendo arquivos CSVs a partir do esquema definido.
* Adicionando valor negativo às transações de saída.

In [ ]:

schema2 = StructType() \
    .add('Id', IntegerType(),True) \
    .add('Cliente_Id', IntegerType(),True) \
    .add('Valor', FloatType(),True) \
    .add('DataHora', TimestampType(),True)

df_transaction_in = spark.read.format('csv') \
    .option('inferSchema', True) \
    .schema(schema2) \
    .load('./data/transaction/in/', delimiter=';')
    
df_transaction_out = spark.read.format('csv') \
    .option('inferSchema', True) \
    .schema(schema2) \
    .load('./data/transaction/out/', delimiter=';')

df_transaction_out.withColumn('Valor', - df_transaction_out['Valor'])
df_transaction_out.show()

* Unindo dataframes de transações.

In [ ]:
def unionAll(dfs):
    return functools.reduce(
        lambda df_transaction_in, 
        df_transaction_out: 
            df_transaction_in.union(df_transaction_out.select(df_transaction_in.columns)), dfs)

df_transaction = unionAll([df_transaction_in, df_transaction_out])
df_transaction.show()

### Normalização dos dados:

* Tranformar todos os dados em lower case. 
* Limpar espaços em branco desnecessários.
* Retirar qualquer linha com dados que não se encaixam no esquema definido (cabeçalhos lidos como dados).

In [ ]:
df_clients = df_clients.withColumn('Nome', F.lower(df_clients['Nome']))

df_clients = df_clients.withColumn('Nome', F.trim(df_clients.Nome))

df_clients = df_clients.na.drop('any')

df_transaction = df_transaction.na.drop('all')

### Adicionando colunas:

* Criando coluna 'DDD' e 'Country_code' a partir dos dados da coluna 'Telefone'.

In [ ]:
df_clients = df_clients \
    .withColumn('DDD', F.substring('Telefone', 5, 2)) \
    .withColumn('Country_code', F.substring('Telefone', 1, 3))

df_clients.show()
print(df_clients.count())

* Criando colunas 'Hora', 'Minuto', 'Segundo' e 'Dia' a partir dos dados da coluna 'DataHora'. 

In [ ]:
df_transaction = df_transaction \
    .withColumn('Hora', F.hour(F.col('DataHora'))) \
    .withColumn('Minuto', F.minute(F.col('DataHora'))) \
    .withColumn('Segundo', F.second(F.col('DataHora'))) \
    .withColumn('Dia', F.to_date(F.col('DataHora'))) \

df_transaction.show()
print(df_transaction.count())

### Criar csv com dados tratados:
* Salvar em único csv todos os dados de clientes tratados na pasta 'clients_clean':

In [ ]:
df_clients.coalesce(1).write.option('header','true') \
    .mode('overwrite') \
    .csv('./data/clients/clients_clean')

* Salvar em único csv todos os dados de transações tratados na pasta 'transaction_clean':

In [ ]:
df_transaction.coalesce(1).write.option('header','true') \
    .mode('overwrite') \
    .csv('./data/transaction/transaction_clean')

### Tentativa de popular tabelas no SQL server via pyspark:
* Não foi concluída pois não encontramos o drive específico para tal tarefa. 

In [ ]:
# import os

# server = os.environ['SERVER']
# database = os.environ['DATABASE']
# username = os.environ['USERNAME']
# password = os.environ['PASSWORD']

# df_clients.write \
#   .format("com.microsoft.sqlserver.jdbc.spark") \
#   .mode("append") \
#   .option("url", "jdbc:sqlserver://anti-fraude.database.windows.net:1433;databaseName=Anti_fraude_s_a;") \
#   .option("dbtable", "clientes") \
#   .option("user", username) \
#   .option("password", password) \
#   .option("encrypt", True) \
#   .option("trustServerCertificate", False) \
#   .option("hostNameInCertificate", '*database.windows.net') \
#   .option("loginTimeout", 30) \
#   .save()